In [1]:
import collections
import os
import string
import sys

import pandas as pd
from nltk import word_tokenize
from nltk.corpus import stopwords
from pprint import pprint
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Nico\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
data_path = "../data/hopitaux/all/"

# Choix d'une décennie et du nombre de clusters

In [4]:
DECADE = ''
N_CLUSTERS = 5

# Chargement des fichiers de la décennie

In [5]:
files = [f for f in sorted(os.listdir(data_path)) if f"_{DECADE[:-1]}" in f]

In [6]:
# Exemple de fichiers
files[:5]

['Bxl_1848_Tome_I1_Part_1.txt',
 'Bxl_1848_Tome_I1_Part_2.txt',
 'Bxl_1907_Tome_I2_Part_1.txt',
 'Bxl_1907_Tome_I2_Part_10.txt',
 'Bxl_1907_Tome_I2_Part_15.txt']

In [7]:
texts = [open(data_path + f, encoding='utf-8').read() for f in files]

In [8]:
# Exemple de textes
texts[0][:400]

"V I L L E DE\n\nBRUXELLES.\n\nbulletin îreô 6e'ûitce0.\n\nDU\n\nCONSEIL COMMUNAL,\n\nANNÉE\n\n1848.\n\n\x0cCI\n\nII\n\n\x0cV I L L E DE B R U X E L L E S .\n\nDtl\n\nCONSEIL COMMUNAL.\nANNÉE\n\n1848.\n\nBRUXELLES,\nIMPRIMERIE\n\nD E J. H . B R I A R D ,\n\nBUE NEUVE, 31, FAUBOURG DE NAJIIIR.\n\n1849\n\n\x0cDE\n\npublics: 1\nUl de la «\nde Véglîst\n\nLeCoi\nUheoi\n\nproeis-v\nM.le\n\nvantes :\nNT\nstase\npoiirinl\ni l . li\ndu Cons\nm o t al\nM. if\nvantes :\nn\n«"

# Vectorisation du texte

In [9]:
def process_text(text, stem=True):
    """ Tokenize text and remove punctuation """
    text = text.translate(string.punctuation)
    tokens = word_tokenize(text)
    return tokens

In [10]:
vectorizer = TfidfVectorizer(tokenizer=process_text,
                                stop_words=stopwords.words('french'),
                                max_df=0.5,
                                min_df=0.1,
                                lowercase=True)

In [11]:
%time tfidf_vectors = vectorizer.fit_transform(texts)

Wall time: 25.4 s


In [12]:
tfidf_vectors

<152x6544 sparse matrix of type '<class 'numpy.float64'>'
	with 211562 stored elements in Compressed Sparse Row format>

In [13]:
# Exemple de vecteur TFIDF
pd.Series(
    tfidf_vectors[0].toarray()[0],
    index=vectorizer.get_feature_names()
    ).sort_values(ascending=False)

conseils        0.355202
décret          0.291067
hospices        0.289424
ouvriers        0.233620
l'honorable     0.200891
                  ...   
maatschappij    0.000000
machines        0.000000
machtiging      0.000000
madame          0.000000
#               0.000000
Length: 6544, dtype: float64

# Clustering des vecteurs TFIDF

In [14]:
km_model = KMeans(n_clusters=N_CLUSTERS)

In [15]:
km_model.fit(tfidf_vectors)

KMeans(n_clusters=5)

In [16]:
clustering = collections.defaultdict(list)

for idx, label in enumerate(km_model.labels_):
    clustering[label].append(files[idx])

In [17]:
pprint(dict(clustering))

{0: ['Bxl_1935_Tome_II_Part_1.txt',
     'Bxl_1935_Tome_II_Part_10.txt',
     'Bxl_1935_Tome_II_Part_2.txt',
     'Bxl_1935_Tome_II_Part_3.txt',
     'Bxl_1935_Tome_II_Part_4.txt',
     'Bxl_1935_Tome_II_Part_5.txt',
     'Bxl_1935_Tome_II_Part_7.txt',
     'Bxl_1935_Tome_II_Part_9.txt',
     'Bxl_1935_Tome_I_Part_1.txt',
     'Bxl_1935_Tome_I_Part_2.txt',
     'Bxl_1935_Tome_I_Part_3.txt',
     'Bxl_1935_Tome_I_Part_4.txt',
     'Bxl_1935_Tome_I_Part_5.txt',
     'Bxl_1935_Tome_I_Part_6.txt',
     'Bxl_1935_Tome_I_Part_7.txt',
     'Bxl_1935_Tome_I_Part_8.txt'],
 1: ['Bxl_1970_Tome_I1_Part_1.txt',
     'Bxl_1970_Tome_I1_Part_12.txt',
     'Bxl_1970_Tome_I1_Part_13.txt',
     'Bxl_1970_Tome_I1_Part_2.txt',
     'Bxl_1970_Tome_I1_Part_3.txt',
     'Bxl_1970_Tome_I1_Part_4.txt',
     'Bxl_1970_Tome_I1_Part_5.txt',
     'Bxl_1970_Tome_I1_Part_6.txt',
     'Bxl_1970_Tome_I1_Part_7.txt',
     'Bxl_1970_Tome_I1_Part_8.txt',
     'Bxl_1970_Tome_I1_Part_9.txt',
     'Bxl_1970_Tome_I2_Part_1.tx